In [185]:
import json
import numpy as np

In [186]:
def crear_tf_idf(inverted_index: dict[dict[int]], 
                 tf_log_scale: bool = True) -> tuple[dict[dict[float]], list[str], list[int]]:
    """
    Calcula la matriz TF-IDF a partir de un índice invertido.

    Esta función toma un índice invertido que mapea términos a documentos y sus frecuencias, 
    y calcula la matriz TF-IDF para cada término en cada documento. La opción de usar escala 
    logarítmica para la frecuencia de términos (TF) es configurable.

    Args:
        inverted_index (dict[dict[int]]): 
            Un diccionario donde las claves son términos (str) y los valores son diccionarios.
            Estos diccionarios internos mapean el ID del documento (int) a la frecuencia de ese 
            término en el documento.

        tf_log_scale (bool, opcional): 
            Indica si se debe aplicar escala logarítmica al cálculo de la frecuencia de términos (TF).
            Por defecto es True, lo que significa que se usará la fórmula `log10(1 + frecuencia)`. Si 
            se establece en False, se usará la frecuencia sin escala logarítmica.

    Returns:
        tuple[dict[dict[float]], list[str], list[int]]: 
            Retorna una tupla que contiene:
            - tf_idf (dict[dict[float]]): Un diccionario donde las claves son IDs de documentos 
              y los valores son diccionarios. Estos diccionarios internos mapean términos a sus 
              correspondientes valores TF-IDF en ese documento.
            - terms (list[str]): Una lista ordenada de los términos presentes en el índice invertido.
            - docs (list[int]): Una lista ordenada de los IDs de documentos únicos en el corpus.
    """

    # Obtiene y ordena los términos únicos en el índice invertido.
    terms = sorted(list(inverted_index.keys()))

    # Extrae y ordena los IDs de documentos únicos en el corpus.
    docs = {doc for docs_freq in inverted_index.values() for doc in docs_freq}
    docs = sorted(list(docs))

    # Calcula el Inverse Document Frequency (IDF) para cada término.
    # IDF = log10(N / df), donde N es el número total de documentos y df es la frecuencia de documentos que contienen el término.
    idf = {term: np.log10(len(docs) / len(doc_freq))
           for term, doc_freq in inverted_index.items()}

    # Calcula el Term Frequency (TF) para cada término en cada documento.
    # Si se aplica escala logarítmica, se usa la fórmula: TF = log10(1 + frecuencia).
    # Si no, se utiliza la frecuencia directa (método empleado por Gensim).
    if tf_log_scale:
        tf = {doc: {term: np.log10(
            1 + inverted_index[term].get(doc, 0)) for term in inverted_index.keys()} for doc in docs}
    else:
        tf = {doc: {term: inverted_index[term].get(
            doc, 0) for term in inverted_index.keys()} for doc in docs}

    # Calcula el TF-IDF para cada término en cada documento.
    # TF-IDF = TF * IDF para cada término en cada documento.
    tf_idf = {doc: {term: tf[doc][term] * idf[term]
                    for term in terms} for doc in docs}

    # Retorna la matriz TF-IDF, la lista de términos y la lista de documentos.
    return tf_idf, terms, docs

In [187]:
inverted_index = json.loads(open('./output/inverted_index.json').read())

In [193]:
tf_idf, terms, docs = crear_tf_idf(inverted_index, 
                                   tf_log_scale=True)

In [194]:
tf_idf_matrix = np.array([[tf_idf[doc][term] for term in terms] for doc in docs])

# normalize the tf-idf matrix by row
tf_idf_matrix = tf_idf_matrix / np.linalg.norm(tf_idf_matrix, axis=1)[:, None]

tf_idf_matrix.shape 

(331, 12463)

In [195]:
tf_idf_matrix[0][55]

0.03050493640437181